<a href="https://colab.research.google.com/github/TSGreenwood/tsgreenwood.github.io/blob/google_cloud/bike_routes_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Analysis and Preprocessing of TNETBikeNetwork.gpkg

This dataset is too big to host on github or google drive, so I hosted in in google cloud.

In [14]:
from google.colab import auth
auth.authenticate_user()

Setting GCS Project and my Google Cloud project ID

In [17]:
!gcloud config set project 'taragreenwoodpersonalwebsite'

Updated property [core/project].


In [ ]:
#!pip install gcsfs geopandas

Bring hefty dataset in for geopandas manipulation

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import geopandas as gpd

In [18]:
#Info about this dataset here: https://gis-kingcounty.opendata.arcgis.com/datasets/fe12aabfcd704a0998f16715a81ccd43_1458/about and here:https://www5.kingcounty.gov/sdc/FGDCDocs/TRANS_NETWORK_BIKE_faq.htm
bike_routes_gdf = gpd.read_file('gs://kingcountypublictransit/data/TNETBikeNetwork.gpkg')

Info about this dataset here: https://gis-kingcounty.opendata.arcgis.com/datasets/fe12aabfcd704a0998f16715a81ccd43_1458/about and here:https://www5.kingcounty.gov/sdc/FGDCDocs/TRANS_NETWORK_BIKE_faq.htm

In [ ]:
bike_routes_gdf.info()
bike_routes_gdf.head()

In [ ]:
print(bike_routes_gdf[bike_routes_gdf.duplicated()])

In [ ]:
bike_routes_gdf.plot()

In [23]:
#This dataset has too much unnecessary information so we'll reduce dimensions
bike_columns_to_drop = ['OBJECTID','CREATEDATE','DB_MD_DATE','END_DATE','CFCC_ID','GRADE','PREFIX_L','NAME_L',
                        'TYPE_L','SUFFIX_L','PREFIX_R','NAME_R','TYPE_R','SUFFIX_R','FULLNAME_L','AP_NAME_L','AP_NAME_R','A_NAME_L',
                        'A_NAME_R','JURIS_L','JURIS_R','CITY_L','CITY_R','ZIP_L','ZIP_R']
bike_routes_gdf = bike_routes_gdf.drop(columns=bike_columns_to_drop)
bike_routes_gdf['TLINK_ID'] = bike_routes_gdf['TLINK_ID'].astype(str)
bike_routes_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 143876 entries, 0 to 143875
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   TLINK_ID    143876 non-null  object  
 1   EFFEC_DATE  143876 non-null  object  
 2   FR_TPOINT   143876 non-null  int64   
 3   TO_TPOINT   143876 non-null  int64   
 4   FULLNAME_R  143876 non-null  object  
 5   ROLL_LEN    143876 non-null  float64 
 6   Shape_Leng  143876 non-null  float64 
 7   geometry    143876 non-null  geometry
dtypes: float64(2), geometry(1), int64(2), object(3)
memory usage: 8.8+ MB


In [30]:
# Write this to geojson

bike_routes_gdf.to_file('bike_lanes.geojson', driver='GeoJSON')

In [31]:
# Export this minty fresh geodata to my Google Cloud bucket

bucket_name = 'kingcountypublictransit'

# Upload the file using gsutil
!gsutil cp ./bike_lanes.geojson gs://kingcountypublictransit/data/

Copying file://./bike_lanes.geojson [Content-Type=application/geo+json]...
\
Operation completed over 1 objects/51.2 MiB.                                     
